In [286]:
import numpy as np
import tensorflow as tf
import math
import re
import string
import os

import nltk
import pandas as pd
from langdetect import detect
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


In [287]:
new_model = tf.keras.models.load_model('saved_model/my_model.h5')

# Check its architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               5120512   
                                                                 
 dense_1 (Dense)             (None, 5)                 2565      
                                                                 
Total params: 5123077 (19.54 MB)
Trainable params: 5123077 (19.54 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Model: "sequential_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_68 (Dense)                │ (None, 100)            │     1,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_70 (Dense)                │ (None, 5)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,005,407 (3.84 MB)

 Trainable params: 1,005,405 (3.84 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [288]:
data = pd.read_csv('Datafiniti_Hotel_Reviews.csv', encoding='ISO-8859-1', sep=',')




In [289]:
data = data[[ 'reviews.text', 'reviews.title', 'reviews.rating']]
data['reviews.text'] = data['reviews.title'] + " " + data['reviews.text']

In [290]:
data = data.drop_duplicates()
data = data.dropna()
data.shape

(9995, 3)

In [291]:
import matplotlib.pyplot as plt

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "error"


data['lang'] = data['reviews.text'].apply(lambda x: detect_lang(x))
data = data.loc[data['lang'] == 'en']
data['lang'].value_counts()
data.shape

(9888, 4)

In [292]:
nltk.download('stopwords')
en_stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()


def clean_text(text):
    result = "".join([word for word in text if word not in string.punctuation])
    tokens = re.split('\W+', result)
    text = [ps.stem(word) for word in tokens if word not in en_stopwords]
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/timothee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [293]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
dataRating = data['reviews.rating']
data = data.drop(columns=['reviews.rating'])
dataRating = dataRating.apply(lambda x: x - 5 if x > 5 else x)
dataRating = dataRating.apply(lambda x: math.floor(x))
dataRating

0       5
1       5
2       5
3       2
4       5
       ..
9995    3
9996    4
9997    4
9998    1
9999    1
Name: reviews.rating, Length: 9888, dtype: int64

In [294]:

vectorizer = TfidfVectorizer(max_features=10000, analyzer=clean_text)


X_train_vectorized = vectorizer.fit_transform(data['reviews.text'])

X_train_vectorized = X_train_vectorized.todense()

In [295]:

np.argmax(new_model.predict(X_train_vectorized), axis=1) + 1

309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step


array([5, 5, 5, ..., 5, 5, 5])

In [296]:
dataRating

0       5
1       5
2       5
3       2
4       5
       ..
9995    3
9996    4
9997    4
9998    1
9999    1
Name: reviews.rating, Length: 9888, dtype: int64

In [297]:
#calculate the accuracy
from sklearn.metrics import accuracy_score
accuracy_score(dataRating, np.argmax(new_model.predict(X_train_vectorized), axis=1) + 1)

309/309 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step


0.3368729773462783